PLZ→AGS ist in Deutschland oft 1:n. Viele PLZ decken mehrere Gemeinden ab. Ohne zusätzliche Filter (Kreis/Bundesland/Ortsname) bleiben Matches stecken.

Namensvarianten (z. B. “Kiel, Landeshauptstadt” vs. “Kiel”) und Sonderzeichen/Bindestriche killen einfache Vergleiche.

regio-Felder uneinheitlich: regio3 kann Stadtteil/Ortsteil statt Gemeinde sein; geo_krs/regio2 enthalten teils Namen statt Schlüssel.

In [13]:
import pandas as pd

df = pd.read_csv("../data/immo_data.csv")
print(df.shape)
df.head()


(268850, 49)


,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,...,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,NaN,False,False,6,4.62,10.0,...,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,NaN,NaN,NaN,NaN,NaN,May19
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,NaN,False,True,8,3.47,10.0,...,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,NaN,NaN,NaN,2019.0,NaN,NaN,May19
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,True,True,8,2.72,2.4,...,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",NaN,NaN,NaN,NaN,NaN,Oct19
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,NaN,False,True,9,1.53,40.0,...,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,NaN,87.23,NaN,NaN,NaN,NaN,May19
4,Bremen,138.00,self_contained_central_heating,NaN,NaN,False,True,19,2.46,NaN,...,Bremen,Neu_Schwachhausen,Es handelt sich hier um ein saniertes Mehrfami...,Diese Wohnung wurde neu saniert und ist wie fo...,NaN,NaN,NaN,NaN,NaN,Feb20


In [14]:
# for col in ["garden", "lift", "hasKitchen"]:
#     if col in df.columns:
#         df[col] = df[col].astype(bool).astype(int)


In [15]:
# num_cols = ["rooms", "area_sqm", "floor", "year_built", "numberOfFloors", "rent_cold"]
# for col in num_cols:
#     if col in df.columns:
#         df[col] = pd.to_numeric(df[col], errors="coerce")


In [16]:
import pandas as pd
import numpy as np

# 0) Einlesen
df = pd.read_csv("../data/immo_data.csv")
print("Rows, Cols:", df.shape)
print("Erste 49 Spalten:", list(df.columns)[:49])
print("\nDtypes:\n", df.dtypes.head(20))

# 1) Bool-Spalten -> 0/1
#   (nimmt echte bools + typische Textwerte wie 'true/false', 'ja/nein', 'yes/no', '1/0')
def to_bool01(series: pd.Series) -> pd.Series:
    if series.dtype == "bool":
        return series.astype(int)
    s = series.astype("string").str.strip().str.lower()
    mapping = {
        "true": 1, "false": 0,
        "ja": 1, "nein": 0,
        "yes": 1, "no": 0,
        "y": 1, "n": 0,
        "t": 1, "f": 0,
        "1": 1, "0": 0
    }
    return s.map(mapping)

# Kandidaten (werden nur umgewandelt, wenn vorhanden)
bool_candidates = [
    "balcony", "garden", "lift", "hasKitchen",
    "cellar", "petsAllowed"  # nur falls in den Daten
]
# plus alle echten bool-Dtypes
bool_candidates += [c for c in df.select_dtypes(include=["bool"]).columns]
bool_candidates = list(dict.fromkeys(bool_candidates))  # uniq

converted = []
for c in bool_candidates:
    if c in df.columns:
        try:
            df[c] = to_bool01(df[c])
            converted.append(c)
        except Exception as e:
            print(f"!! Konnte {c} nicht in 0/1 umwandeln:", e)

print("In 0/1 umgewandelt:", converted)
if "balcony" in df.columns:
    print("balcony Beispiel:", df["balcony"].head().tolist(), "| dtype:", df["balcony"].dtype)



Rows, Cols: (268850, 49)
Erste 49 Spalten: ['regio1', 'serviceCharge', 'heatingType', 'telekomTvOffer', 'telekomHybridUploadSpeed', 'newlyConst', 'balcony', 'picturecount', 'pricetrend', 'telekomUploadSpeed', 'totalRent', 'yearConstructed', 'scoutId', 'noParkSpaces', 'firingTypes', 'hasKitchen', 'geo_bln', 'cellar', 'yearConstructedRange', 'baseRent', 'houseNumber', 'livingSpace', 'geo_krs', 'condition', 'interiorQual', 'petsAllowed', 'street', 'streetPlain', 'lift', 'baseRentRange', 'typeOfFlat', 'geo_plz', 'noRooms', 'thermalChar', 'floor', 'numberOfFloors', 'noRoomsRange', 'garden', 'livingSpaceRange', 'regio2', 'regio3', 'description', 'facilities', 'heatingCosts', 'energyEfficiencyClass', 'lastRefurbish', 'electricityBasePrice', 'electricityKwhPrice', 'date']

Dtypes:
 regio1                       object
serviceCharge               float64
heatingType                  object
telekomTvOffer               object
telekomHybridUploadSpeed    float64
newlyConst                     bool

In [17]:
# Missing-Werte je Spalte (Anzahl & Prozent), absteigend sortiert
n = len(df)
missing_cnt = df.isna().sum()
missing_pct = (missing_cnt / n * 100).round(1)

miss_summary = (
    pd.DataFrame({
        "missing_count": missing_cnt,
        "missing_pct": missing_pct,
        "dtype": df.dtypes
    })
    .sort_values("missing_count", ascending=False)
)

print(f"Zeilen gesamt: {n}\n")
print("Top 30 Spalten mit Missing-Werten:")
print(miss_summary[miss_summary["missing_count"] > 0].head(30).to_string())

# Optional: komplette Liste in eine CSV schreiben, wenn du sie in Ruhe ansehen willst
miss_summary.to_csv("../data/immo_missing_summary.csv", encoding="utf-8")
print("\nGespeichert → ../data/immo_missing_summary.csv")


Zeilen gesamt: 268850

Top 30 Spalten mit Missing-Werten:
                          missing_count  missing_pct    dtype
telekomHybridUploadSpeed         223830         83.3  float64
electricityBasePrice             222004         82.6  float64
electricityKwhPrice              222004         82.6  float64
petsAllowed                      206543         76.8  float64
energyEfficiencyClass            191063         71.1   object
lastRefurbish                    188139         70.0  float64
heatingCosts                     183332         68.2  float64
noParkSpaces                     175798         65.4  float64
interiorQual                     112665         41.9   object
thermalChar                      106506         39.6  float64
numberOfFloors                    97732         36.4  float64
houseNumber                       71018         26.4   object
streetPlain                       71013         26.4   object
condition                         68489         25.5   object
yearConstruc

In [43]:
# heatingType bereinigen
if "heatingType" in df.columns:
    # Seltene Werte zusammenfassen
    top_values = ["central_heating", "district_heating", "gas_heating"]
    df["heatingType_clean"] = df["heatingType"].where(
        df["heatingType"].isin(top_values), "other"
    )
    # Missing füllen
    df["heatingType_clean"] = df["heatingType_clean"].fillna("unknown")


In [64]:
# Spalten, die wir rauswerfen wollen
drop_cols = [
    "telekomHybridUploadSpeed",
    "electricityBasePrice",
    "electricityKwhPrice",
    "petsAllowed",
    "energyEfficiencyClass",
    "lastRefurbish",
    "heatingCosts",
    "noParkSpaces",
    "interiorQual",
    "thermalChar",
    "numberOfFloors",
    "houseNumber",
    "streetPlain",
    "condition",
    "telekomUploadSpeed",
    "telekomTvOffer",
    "electricityKwhPrice",
    "electricityBasePrice",
    "date",
    "street",
    "description",
    "facilities",
    "scoutId",
    "noRoomsRange",
    "newlyConst",
    "livingSpaceRange",
    "yearConstructedRange",
    "picturecount",
    "heatingType"
]

print("Vorher:", df.shape)
df = df.drop(columns=[c for c in drop_cols if c in df.columns])
print("Nachher:", df.shape)


Vorher: (268850, 26)
Nachher: (268850, 25)


In [48]:
import os

# 1) Zielordner sicherstellen
out_dir = "../data/clean"
os.makedirs(out_dir, exist_ok=True)

# 2) Pfad festlegen
out_path = f"{out_dir}/immo_clean3.csv"

# 3) Speichern
df.to_csv(out_path, index=False, encoding="utf-8")

print(f"Gespeichert → {out_path}")
print("Shape:", df.shape)
print("Spalten:", df.columns.tolist())


Gespeichert → ../data/clean/immo_clean3.csv
Shape: (268850, 30)
Spalten: ['regio1', 'serviceCharge', 'heatingType', 'newlyConst', 'balcony', 'picturecount', 'pricetrend', 'totalRent', 'yearConstructed', 'firingTypes', 'hasKitchen', 'geo_bln', 'cellar', 'yearConstructedRange', 'baseRent', 'livingSpace', 'geo_krs', 'lift', 'baseRentRange', 'typeOfFlat', 'geo_plz', 'noRooms', 'floor', 'garden', 'livingSpaceRange', 'regio2', 'regio3', 'heatingType_clean', 'dekade', 'periode_0005']


In [45]:
print("Vor dem Speichern:", df.shape)
print(df.columns.tolist())


Vor dem Speichern: (268850, 31)
['regio1', 'serviceCharge', 'heatingType', 'newlyConst', 'balcony', 'picturecount', 'pricetrend', 'totalRent', 'yearConstructed', 'firingTypes', 'hasKitchen', 'geo_bln', 'cellar', 'yearConstructedRange', 'baseRent', 'livingSpace', 'geo_krs', 'lift', 'baseRentRange', 'typeOfFlat', 'geo_plz', 'noRooms', 'floor', 'noRoomsRange', 'garden', 'livingSpaceRange', 'regio2', 'regio3', 'heatingType_clean', 'dekade', 'periode_0005']


In [32]:
import numpy as np
if "yearConstructed" in df.columns:
    yr = pd.to_numeric(df["yearConstructed"], errors="coerce")
    df["dekade"] = (np.floor(yr / 10) * 10).astype("Int64")  # z.B. 1990, 2000, …


In [39]:
df

,regio1,serviceCharge,heatingType,newlyConst,balcony,picturecount,pricetrend,totalRent,yearConstructed,scoutId,...,noRooms,floor,noRoomsRange,garden,livingSpaceRange,regio2,regio3,heatingType_clean,dekade,periode_0005
0,Nordrhein_Westfalen,245.0,central_heating,0,0,6,4.62,840.0,1965.0,96107057,...,4.0,1.0,4,1,4,Dortmund,Schüren,central_heating,1960,1960_1969
1,Rheinland_Pfalz,134.0,self_contained_central_heating,0,1,8,3.47,NaN,1871.0,111378734,...,3.0,NaN,3,0,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,other,1870,vor_1919
2,Sachsen,255.0,floor_heating,1,1,8,2.72,1300.0,2019.0,113147523,...,3.0,3.0,3,0,4,Dresden,Äußere_Neustadt_Antonstadt,other,2010,2016_plus
3,Sachsen,58.15,district_heating,0,1,9,1.53,NaN,1964.0,108890903,...,3.0,3.0,3,0,2,Mittelsachsen_Kreis,Freiberg,district_heating,1960,1960_1969
4,Bremen,138.0,self_contained_central_heating,0,1,19,2.46,903.0,1950.0,114751222,...,3.0,1.0,3,0,4,Bremen,Neu_Schwachhausen,other,1950,1950_1959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268845,Bayern,90.0,heat_pump,0,1,0,2.74,910.0,2016.0,115641081,...,3.0,NaN,3,0,4,Weilheim_Schongau_Kreis,Eberfing,other,2010,2016_plus
268846,Hessen,220.0,gas_heating,0,1,12,6.49,1150.0,1983.0,96981497,...,3.5,1.0,3,0,5,Bergstraße_Kreis,Viernheim,gas_heating,1980,1980_1989
268847,Hessen,220.0,central_heating,0,1,21,2.90,930.0,1965.0,66924271,...,4.0,1.0,4,1,4,Limburg_Weilburg_Kreis,Limburg_an_der_Lahn,central_heating,1960,1960_1969
268848,Nordrhein_Westfalen,175.0,heat_pump,1,1,16,4.39,1015.0,2019.0,110938302,...,2.0,NaN,2,0,3,Köln,Dellbrück,other,2010,2016_plus


In [36]:
def jahr_to_periode(y):
    if pd.isna(y): 
        return np.nan
    y = int(y)
    if y < 1919:        return "vor_1919"
    if 1919 <= y <= 1949: return "1919_1949"
    if 1950 <= y <= 1959: return "1950_1959"
    if 1960 <= y <= 1969: return "1960_1969"
    if 1970 <= y <= 1979: return "1970_1979"
    if 1980 <= y <= 1989: return "1980_1989"
    if 1990 <= y <= 1999: return "1990_1999"
    if 2000 <= y <= 2009: return "2000_2009"
    if 2010 <= y <= 2015: return "2010_2015"
    if y >= 2016:        return "2016_plus"
    return np.nan

# yearConstructed -> periode_0005
if "yearConstructed" in df.columns:
    df["yearConstructed"] = pd.to_numeric(df["yearConstructed"], errors="coerce")
    df["periode_0005"] = df["yearConstructed"].apply(jahr_to_periode)
    print(df["periode_0005"].value_counts(dropna=False).head(10))


periode_0005
NaN          57045
vor_1919     34616
2016_plus    28685
1990_1999    26664
1960_1969    25563
1970_1979    25176
1980_1989    18826
1950_1959    18437
1919_1949    17692
2010_2015     8399
Name: count, dtype: int64


In [38]:
# serviceCharge -> numerisch + fehlende = 0
if "serviceCharge" in df.columns:
    # Komma als Dezimalpunkt erlauben
    sc = pd.to_numeric(
        df["serviceCharge"].astype("string").str.replace(",", ".", regex=False),
        errors="coerce"
    )
    df["serviceCharge"] = sc.fillna(0.0)

    print("serviceCharge: fehlende nach Füllung →", df["serviceCharge"].isna().sum())
    print("serviceCharge: min/max/mean →",
          df["serviceCharge"].min(), df["serviceCharge"].max(), round(df["serviceCharge"].mean(), 2))


serviceCharge: fehlende nach Füllung → 0
serviceCharge: min/max/mean → 0.0 146118.0 147.32


In [51]:
import numpy as np
import pandas as pd

# 1) Numerisch sicherstellen (Komma -> Punkt erlauben)
def _to_num(s):
    return pd.to_numeric(s.astype("string").str.replace(",", ".", regex=False), errors="coerce")

for col in ["baseRent", "serviceCharge", "totalRent"]:
    if col in df.columns:
        df[col] = _to_num(df[col])

# 2) fehlende totalRent berechnen: baseRent + serviceCharge
if {"baseRent", "serviceCharge", "totalRent"}.issubset(df.columns):
    # Falls serviceCharge/ baseRent fehlen, wie 0 behandeln
    calc_total = df["baseRent"].fillna(0) + df["serviceCharge"].fillna(0)

    # Nur dort überschreiben, wo totalRent fehlt oder 0 ist
    mask = df["totalRent"].isna() | (df["totalRent"] == 0)
    n_before = int(mask.sum())
    df.loc[mask, "totalRent"] = calc_total.loc[mask]
    print(f"totalRent ergänzt in {n_before} Zeilen.")

    # Optional: einfache Plausi (totalRent >= baseRent)
    bad = (df["totalRent"] < df["baseRent"]).sum()
    if bad:
        print(f"Warnung: {bad} Zeilen mit totalRent < baseRent gefunden.")

# 3) (Optional) einmal zusammenfassen
print(df[["baseRent", "serviceCharge", "totalRent"]].describe())


totalRent ergänzt in 40753 Zeilen.
Warnung: 215 Zeilen mit totalRent < baseRent gefunden.
          baseRent  serviceCharge     totalRent
count     268850.0       268850.0      268850.0
mean    694.129432     147.320366    925.831709
std    19536.01758     305.247775  36199.002823
min            0.0            0.0           0.0
25%          338.0           90.0         465.0
50%          490.0          131.0         647.0
75%          799.0          185.0         980.0
max      9999999.0       146118.0    15751535.0


In [61]:
# --- 1) firingTypes & typeOfFlat sauber machen und als Kategorie casten ---
for col in ["firingTypes", "typeOfFlat"]:
    if col in df.columns:
        # zu String, trimmen, klein schreiben
        s = df[col].astype("string").str.strip().str.lower()
        # häufige Schreibvarianten etwas vereinheitlichen (optional)
        s = (s
             .str.replace(r"\s+", " ", regex=True)   # Mehrfachspaces -> 1 Space
             .str.replace("/", ",", regex=False)     # / -> ,
             .str.replace(";", ",", regex=False))    # ; -> ,
        # fehlende als 'unknown'
        s = s.fillna("unknown")
        df[col] = s.astype("category")

# kurze Sichtprüfung:
print("firingTypes (Top 10):")
print(df["firingTypes"].value_counts(dropna=False).head(15))
print("\ntypeOfFlat (Top 10):")
print(df["typeOfFlat"].value_counts(dropna=False).head(15))
print()



firingTypes (Top 10):
firingTypes
gas                                     110899
unknown                                  56964
district_heating                         49389
oil                                      18137
natural_gas_light                        10080
electricity                               4838
natural_gas_heavy                         4542
pellet_heating                            2478
geothermal                                2442
gas:electricity                           1354
local_heating                              932
environmental_thermal_energy               900
combined_heat_and_power_fossil_fuels       863
steam_district_heating                     787
liquid_gas                                 451
Name: count, dtype: int64

typeOfFlat (Top 10):
typeOfFlat
apartment              131522
unknown                 36614
roof_storey             34787
ground_floor            31538
other                    9519
maisonette               9319
raised_ground_floor   

In [57]:
# --- 2) Umbenennen für Training ---
rename_map = {
    "baseRent": "rent_cold",
    "livingSpace": "area_sqm",
    "noRooms": "rooms",
}
df = df.rename(columns=rename_map)

df

,regio1,serviceCharge,heatingType,balcony,pricetrend,totalRent,yearConstructed,firingTypes,hasKitchen,geo_bln,...,typeOfFlat,geo_plz,rooms,floor,garden,regio2,regio3,heatingType_clean,dekade,periode_0005
0,Nordrhein_Westfalen,245.0,central_heating,0,4.62,840.0,1965.0,oil,0,Nordrhein_Westfalen,...,ground_floor,44269,4.0,1.0,1,Dortmund,Schüren,central_heating,1960,1960_1969
1,Rheinland_Pfalz,134.0,self_contained_central_heating,1,3.47,934.0,1871.0,gas,0,Rheinland_Pfalz,...,ground_floor,67459,3.0,NaN,0,Rhein_Pfalz_Kreis,Böhl_Iggelheim,other,1870,vor_1919
2,Sachsen,255.0,floor_heating,1,2.72,1300.0,2019.0,NaN,0,Sachsen,...,apartment,1097,3.0,3.0,0,Dresden,Äußere_Neustadt_Antonstadt,other,2010,2016_plus
3,Sachsen,58.15,district_heating,1,1.53,401.15,1964.0,district_heating,0,Sachsen,...,other,9599,3.0,3.0,0,Mittelsachsen_Kreis,Freiberg,district_heating,1960,1960_1969
4,Bremen,138.0,self_contained_central_heating,1,2.46,903.0,1950.0,gas,0,Bremen,...,apartment,28213,3.0,1.0,0,Bremen,Neu_Schwachhausen,other,1950,1950_1959
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268845,Bayern,90.0,heat_pump,1,2.74,910.0,2016.0,geothermal,0,Bayern,...,roof_storey,82390,3.0,NaN,0,Weilheim_Schongau_Kreis,Eberfing,other,2010,2016_plus
268846,Hessen,220.0,gas_heating,1,6.49,1150.0,1983.0,gas,1,Hessen,...,apartment,68519,3.5,1.0,0,Bergstraße_Kreis,Viernheim,gas_heating,1980,1980_1989
268847,Hessen,220.0,central_heating,1,2.90,930.0,1965.0,gas,0,Hessen,...,apartment,65552,4.0,1.0,1,Limburg_Weilburg_Kreis,Limburg_an_der_Lahn,central_heating,1960,1960_1969
268848,Nordrhein_Westfalen,175.0,heat_pump,1,4.39,1015.0,2019.0,gas,0,Nordrhein_Westfalen,...,apartment,51069,2.0,NaN,0,Köln,Dellbrück,other,2010,2016_plus


In [58]:
# --- 3) Numerische Typen sicherstellen (falls z. B. Komma als Dezimaltrenner vorkam) ---
def _to_num(s):
    return pd.to_numeric(s.astype("string").str.replace(",", ".", regex=False), errors="coerce")

for col in ["rent_cold", "area_sqm", "rooms", "floor", "yearConstructed", "serviceCharge", "totalRent", "picturecount", "pricetrend"]:
    if col in df.columns:
        df[col] = _to_num(df[col])



In [59]:
# --- 4) Optional: totalRent aus rent_cold + serviceCharge auffüllen, wo fehlt/0 ist ---
if {"rent_cold", "serviceCharge", "totalRent"}.issubset(df.columns):
    calc_total = df["rent_cold"].fillna(0) + df["serviceCharge"].fillna(0)
    mask = df["totalRent"].isna() | (df["totalRent"] == 0)
    print("totalRent ergänzt in", int(mask.sum()), "Zeilen.")
    df.loc[mask, "totalRent"] = calc_total.loc[mask]



totalRent ergänzt in 23 Zeilen.


In [65]:
# --- 5) Speichern ---
out_path = "../data/clean/immo_clean7.csv"
df.to_csv(out_path, index=False, encoding="utf-8")
print("Gespeichert →", out_path)



Gespeichert → ../data/clean/immo_clean7.csv


In [62]:
# --- 6) Kleine Zusammenfassung ---
print("\nShapes & dtypes:")
print(df.shape)
print(df.dtypes[["rent_cold","area_sqm","rooms","firingTypes","typeOfFlat"]])



Shapes & dtypes:
(268850, 26)
rent_cold       Float64
area_sqm        Float64
rooms           Float64
firingTypes    category
typeOfFlat     category
dtype: object


In [66]:
import numpy as np
import pandas as pd

# sicherheitshalber numerisch machen
df["floor"] = pd.to_numeric(df.get("floor"), errors="coerce")

# Werte plausibilisieren
df.loc[df["floor"] < 0, "floor"] = 0
df.loc[df["floor"] > 60, "floor"] = 60  # reines Schutznetz

# häufigster Wert (Modus)
if df["floor"].notna().any():
    floor_mode = df["floor"].mode().iloc[0]
else:
    floor_mode = 0

# fehlende füllen
df["floor"] = df["floor"].fillna(floor_mode)

# hübscher Datentyp (nullable-Integer)
df["floor"] = df["floor"].astype("Int16")

print("floor → Modus:", floor_mode, "| fehlend danach:", df["floor"].isna().sum())


floor → Modus: 1.0 | fehlend danach: 0


In [67]:
df["pricetrend"] = pd.to_numeric(df.get("pricetrend"), errors="coerce")
pricetrend_mean = df["pricetrend"].mean(skipna=True)

df["pricetrend"] = df["pricetrend"].fillna(pricetrend_mean)

print("pricetrend → Mittelwert zum Füllen:", round(pricetrend_mean, 4))
print("fehlend danach:", df["pricetrend"].isna().sum())


pricetrend → Mittelwert zum Füllen: 3.389
fehlend danach: 0


In [68]:
# numerisch machen
df["yearConstructed"] = pd.to_numeric(df.get("yearConstructed"), errors="coerce")

# (optional) grobe Dekade für Analyse
df["dekade"] = (df["yearConstructed"] // 10 * 10).astype("Int32")

# die gleichen Perioden wie zensus_0005_clean verwenden
bins   = [-np.inf, 1918, 1949, 1959, 1969, 1979, 1989, 1999, 2009, 2015, np.inf]
labels = ["vor_1919","1919_1949","1950_1959","1960_1969","1970_1979",
          "1980_1989","1990_1999","2000_2009","2010_2015","2016_plus"]

periode_from_year = pd.cut(df["yearConstructed"], bins=bins, labels=labels, right=True)

# falls die Spalte noch nicht existiert: anlegen
if "periode_0005" not in df.columns:
    df["periode_0005"] = pd.Series(pd.NA, index=df.index, dtype="string")

# 1) wo yearConstructed bekannt → Periode setzen
df.loc[periode_from_year.notna(), "periode_0005"] = periode_from_year.astype("string")

# 2) wo yearConstructed fehlt → "unknown"
df["periode_0005"] = df["periode_0005"].fillna("unknown").astype("category")

print("periode_0005 – Verteilung:")
print(df["periode_0005"].value_counts(dropna=False).head(12))


periode_0005 – Verteilung:
periode_0005
unknown      57045
vor_1919     34616
2016_plus    28685
1990_1999    26664
1960_1969    25563
1970_1979    25176
1980_1989    18826
1950_1959    18437
1919_1949    17692
2010_2015     8399
2000_2009     7747
Name: count, dtype: int64


In [69]:
out_path = "../data/clean/immo_clean8.csv"
df.to_csv(out_path, index=False, encoding="utf-8")
print("Gespeichert →", out_path, "| Form:", df.shape)


Gespeichert → ../data/clean/immo_clean8.csv | Form: (268850, 25)


In [70]:
import pandas as pd

df = pd.read_csv("../data/clean/immo_clean8.csv")

print("Shape:", df.shape)
print("\nSpaltenübersicht:")
print(df.dtypes)

print("\nMissing-Werte pro Spalte:")
print(df.isna().sum().sort_values(ascending=False).head(25))


Shape: (268850, 25)

Spaltenübersicht:
regio1                object
serviceCharge        float64
balcony                int64
pricetrend           float64
totalRent            float64
yearConstructed      float64
firingTypes           object
hasKitchen             int64
geo_bln               object
cellar                 int64
rent_cold            float64
area_sqm             float64
geo_krs               object
lift                   int64
baseRentRange          int64
typeOfFlat            object
geo_plz                int64
rooms                float64
floor                  int64
garden                 int64
regio2                object
regio3                object
heatingType_clean     object
dekade               float64
periode_0005          object
dtype: object

Missing-Werte pro Spalte:
yearConstructed    57045
dekade             57045
regio1                 0
pricetrend             0
serviceCharge          0
totalRent              0
firingTypes            0
hasKitchen          

In [73]:
print("Shape:", df.shape)
print("\nSpaltenübersicht:")
print(df.dtypes)

Shape: (268850, 25)

Spaltenübersicht:
regio1                object
serviceCharge        float64
balcony                int64
pricetrend           float64
totalRent            float64
yearConstructed      float64
firingTypes           object
hasKitchen             int64
geo_bln               object
cellar                 int64
rent_cold            float64
area_sqm             float64
geo_krs               object
lift                   int64
baseRentRange          int64
typeOfFlat            object
geo_plz                int64
rooms                float64
floor                  int64
garden                 int64
regio2                object
regio3                object
heatingType_clean     object
dekade               float64
periode_0005          object
dtype: object


In [72]:
print("\nMissing-Werte pro Spalte:")
print(df.isna().sum().sort_values(ascending=False).head(25))


Missing-Werte pro Spalte:
yearConstructed      57045
dekade               57045
regio1                   0
pricetrend               0
serviceCharge            0
totalRent                0
firingTypes              0
hasKitchen               0
balcony                  0
geo_bln                  0
cellar                   0
area_sqm                 0
rent_cold                0
lift                     0
baseRentRange            0
typeOfFlat               0
geo_krs                  0
geo_plz                  0
rooms                    0
garden                   0
floor                    0
regio2                   0
regio3                   0
heatingType_clean        0
periode_0005             0
dtype: int64


In [74]:
# === GKZ aus regio1/2/3 erstellen und speichern =============================
import re
import pandas as pd
from pathlib import Path

# --- Pfade (ggf. anpassen) ---
IMMO_IN   = "../data/clean/immo_clean8.csv"
Z0004_IN  = "../data/clean/zensus_0004_clean.csv"   # hat GKZ + Gemeindename
OUT_WITH  = "../data/clean/immo_with_gkz.csv"
UNMATCHED = "../data/clean/immo_gkz_unmatched_sample.csv"

# --- 1) Hilfsfunktionen ---
BL_TO_CODE = {
    "schleswig-holstein":"01","hamburg":"02","niedersachsen":"03","bremen":"04",
    "nordrhein-westfalen":"05","hessen":"06","rheinland-pfalz":"07","baden-württemberg":"08",
    "baden-wuerttemberg":"08","bayern":"09","saarland":"10","berlin":"11",
    "brandenburg":"12","mecklenburg-vorpommern":"13","sachsen":"14",
    "sachsen-anhalt":"15","thüringen":"16","thueringen":"16",
}

def de_umlaut(s: str) -> str:
    return (s.replace("ä","ae").replace("ö","oe").replace("ü","ue").replace("ß","ss"))

def clean_name(s: pd.Series) -> pd.Series:
    # string -> normalized ortsname
    s = s.astype("string").fillna("")
    s = s.str.lower().map(de_umlaut)
    s = s.str.replace(r"\(.*?\)", "", regex=True)     # Klammern weg
    s = s.str.split(",").str[0]                       # Alles nach Komma kappen
    s = s.str.replace("-", " ", regex=False)
    s = s.str.replace(r"\b(hansestadt|landeshauptstadt|kreisfreie stadt|amtsfreie stadt|gemeinde|stadt|markt)\b", "", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

def bl_code_from_regio1(s: pd.Series) -> pd.Series:
    s = s.astype("string").str.lower().str.strip()
    # kleine Korrekturen
    s = s.str.replace("baden-württemberg", "baden-württemberg", regex=False)
    s = s.str.replace("thüringen", "thüringen", regex=False)
    return s.map(BL_TO_CODE)

# --- 2) Daten laden ---
df = pd.read_csv(IMMO_IN)
z4 = pd.read_csv(Z0004_IN)

# Falls GKZ versehentlich als Float gespeichert wurde → String
if "GKZ" in z4.columns:
    z4["GKZ"] = z4["GKZ"].astype("string").str.strip()

# --- 3) Zensus-Ortsnamen normalisieren + BL-Code aus GKZ ableiten ---
if "Gemeindename" not in z4.columns:
    raise RuntimeError("In zensus_0004_clean.csv fehlt die Spalte 'Gemeindename'.")

z4["gemeinde_clean"] = clean_name(z4["Gemeindename"])
z4["bl_code"] = z4["GKZ"].str[:2]    # die ersten 2 Ziffern = Bundesland

# nur eindeutige Kombinationen (gemeinde_clean, bl_code) behalten:
dups = z4.duplicated(["gemeinde_clean","bl_code"], keep=False)
z4_unique = z4.loc[~dups, ["gemeinde_clean","bl_code","GKZ"]].copy()

# --- 4) Immo-Orte normalisieren + BL-Code aus regio1 ---
if "regio3" not in df.columns or "regio1" not in df.columns:
    print("⚠️  Erwartet: Spalten 'regio1' und 'regio3' im Immo-Datensatz.")
    print("    Bitte prüfen – ansonsten kann kein Namensabgleich erfolgen.")

df["city_raw"]   = df["regio3"].astype("string")
df["city_clean"] = clean_name(df["city_raw"])
df["bl_code"]    = bl_code_from_regio1(df["regio1"])

# --- 5) Merge (left) auf (gemeinde_clean + bl_code) ---
merged = df.merge(
    z4_unique,
    how="left",
    left_on=["city_clean","bl_code"],
    right_on=["gemeinde_clean","bl_code"],
)

# --- 6) Match-Statistik & Unmatched speichern ---
n_all = len(merged)
n_match = merged["GKZ"].notna().sum()
print(f"Match-Quote GKZ: {n_match}/{n_all} = {n_match/n_all:.1%}")

unmatched = merged[merged["GKZ"].isna()][["regio1","regio2","regio3","city_clean"]].drop_duplicates().head(200)
unmatched.to_csv(UNMATCHED, index=False, encoding="utf-8")
print(f"Beispiele ohne Match → {UNMATCHED}")

# --- 7) Aufräumen: Hilfsspalten entfernen, Region-Spalten optional droppen ---
merged = merged.drop(columns=[c for c in ["gemeinde_clean"] if c in merged.columns])

# Wenn du die Region-Spalten NACH erfolgreicher GKZ-Erstellung löschen willst:
DROP_REGIO = True
if DROP_REGIO:
    drop_cols = [c for c in ["regio1","regio2","regio3","geo_bln","geo_krs","geo_plz","city_raw","city_clean","bl_code"] if c in merged.columns]
    merged = merged.drop(columns=drop_cols)

# --- 8) Speichern ---
Path("../data/clean").mkdir(parents=True, exist_ok=True)
merged.to_csv(OUT_WITH, index=False, encoding="utf-8")
print(f"Gespeichert → {OUT_WITH}")


Match-Quote GKZ: 22507/268850 = 8.4%
Beispiele ohne Match → ../data/clean/immo_gkz_unmatched_sample.csv
Gespeichert → ../data/clean/immo_with_gkz.csv


In [75]:
import re
import pandas as pd
from pathlib import Path

IMMO_IN   = "../data/clean/immo_clean8.csv"
Z0004_IN  = "../data/clean/zensus_0004_clean.csv"
OUT_PATH  = "../data/clean/immo_with_gkz_plus.csv"

# --- Hilfen ---
BL_TO_CODE = {
    "schleswig-holstein":"01","hamburg":"02","niedersachsen":"03","bremen":"04",
    "nordrhein-westfalen":"05","hessen":"06","rheinland-pfalz":"07","baden-württemberg":"08",
    "baden-wuerttemberg":"08","bayern":"09","saarland":"10","berlin":"11",
    "brandenburg":"12","mecklenburg-vorpommern":"13","sachsen":"14",
    "sachsen-anhalt":"15","thüringen":"16","thueringen":"16",
}

def de_umlaut(s: str) -> str:
    return s.replace("ä","ae").replace("ö","oe").replace("ü","ue").replace("ß","ss")

def norm(s: pd.Series) -> pd.Series:
    s = s.astype("string").fillna("").str.lower().map(de_umlaut)
    s = s.str.replace(r"\(.*?\)", "", regex=True)
    s = s.str.replace(",", " ", regex=False)
    s = s.str.replace("-", " ", regex=False)
    s = s.str.replace(r"\b(hansestadt|landeshauptstadt|kreisfreie stadt|amtsfreie stadt|gemeinde|stadt|markt)\b", "", regex=True)
    s = s.str.replace(r"\bst\.\b", "sankt", regex=True)
    s = s.str.replace(r"\ba\.? d\.?\b", "an der", regex=True)  # a. d. → an der
    s = s.str.replace(r"\sam\b", " ", regex=True)
    s = s.str.replace(r"\san der\b", " ", regex=True)
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

def bl_code_from(df_col: pd.Series) -> pd.Series:
    return df_col.astype("string").str.lower().map(BL_TO_CODE)

# --- Daten laden ---
df  = pd.read_csv(IMMO_IN)
z4  = pd.read_csv(Z0004_IN)

# Zensus aufbereiten
z4["GKZ"] = z4["GKZ"].astype("string").str.strip()
z4["gemeinde_clean"] = norm(z4["Gemeindename"])
z4["bl2"] = z4["GKZ"].str[:2]
z4["krs5"] = z4["GKZ"].str[:5]

# Duplikate auf Gemeindeebene drop, damit Merge eindeutig ist
dups = z4.duplicated(["gemeinde_clean","bl2"], keep=False)
z4u = z4.loc[~dups, ["GKZ","gemeinde_clean","bl2"]].copy()

# Kreis- und Landes-Aggregate der Zensus-Miete bauen
z4_vals = pd.read_csv("../data/clean/zensus_0004_clean.csv")
z4_vals["GKZ"] = z4_vals["GKZ"].astype("string")
z4_vals["krs5"] = z4_vals["GKZ"].str[:5]
z4_vals["bl2"]  = z4_vals["GKZ"].str[:2]
# numerische Miete
z4_vals["Miete"] = pd.to_numeric(z4_vals["Miete"], errors="coerce")
krs_avg = z4_vals.groupby("krs5", as_index=False)["Miete"].mean().rename(columns={"Miete":"zensus_miete_kreis"})
bl_avg  = z4_vals.groupby("bl2",  as_index=False)["Miete"].mean().rename(columns={"Miete":"zensus_miete_land"})

# --- Immo-Ortskandidaten ---
df["cand1"] = norm(df.get("regio3", pd.Series(index=df.index, dtype="string")))
df["cand2"] = norm(df.get("regio2", pd.Series(index=df.index, dtype="string")))
df["bl2"]   = bl_code_from(df.get("regio1", pd.Series(index=df.index, dtype="string")))

# 1) Gemeindematch: erst cand1, dann cand2
g1 = df.merge(z4u, how="left", left_on=["cand1","bl2"], right_on=["gemeinde_clean","bl2"])
missing_gkz = g1["GKZ"].isna()
g1.loc[missing_gkz, ["GKZ"]] = df.loc[missing_gkz].merge(
    z4u, how="left", left_on=["cand2","bl2"], right_on=["gemeinde_clean","bl2"]
)["GKZ"].values

# 2) GKZ-Präzision kennzeichnen
g1["gkz_precision"] = pd.NA
g1.loc[g1["GKZ"].notna(), "gkz_precision"] = "gemeinde"

# 3) Kreis-Fallback: wenn GKZ fehlt, versuche über geo_krs (Ziffern extrahieren)
tmp = g1["GKZ"].isna()
df_krs = df.loc[tmp, ["geo_krs"]].astype("string").fillna("")
# Kreis-Schlüssel (5 Ziffern) aus geo_krs ziehen, wenn vorhanden
krs5 = df_krs["geo_krs"].str.extract(r"(\d{5})", expand=False)
g1["krs5"] = pd.NA
g1.loc[tmp, "krs5"] = krs5.values

# Kreis-Miete mergen
g1 = g1.merge(krs_avg, how="left", on="krs5")

# 4) Landes-Fallback
g1 = g1.merge(bl_avg, how="left", on="bl2")

# 5) Finale Zensus-Feature bauen + Präzision setzen
g1["zensus_miete"] = pd.NA
g1.loc[g1["GKZ"].notna(), "zensus_miete"] = g1.loc[g1["GKZ"].notna()].merge(
    z4_vals[["GKZ","Miete"]], how="left", on="GKZ"
)["Miete"].values

use_krs = g1["zensus_miete"].isna() & g1["zensus_miete_kreis"].notna()
g1.loc[use_krs, "zensus_miete"] = g1.loc[use_krs, "zensus_miete_kreis"].values
g1.loc[use_krs & g1["gkz_precision"].isna(), "gkz_precision"] = "kreis"

use_bl = g1["zensus_miete"].isna() & g1["zensus_miete_land"].notna()
g1.loc[use_bl, "zensus_miete"] = g1.loc[use_bl, "zensus_miete_land"].values
g1.loc[use_bl & g1["gkz_precision"].isna(), "gkz_precision"] = "land"

# 6) Match-Statistik
n = len(g1)
p_gem  = (g1["gkz_precision"]=="gemeinde").sum()
p_krs  = (g1["gkz_precision"]=="kreis").sum()
p_land = (g1["gkz_precision"]=="land").sum()
print(f"Treffer Gemeinde: {p_gem}/{n} = {p_gem/n:.1%}")
print(f"Fallback Kreis  : {p_krs}/{n} = {p_krs/n:.1%}")
print(f"Fallback Land   : {p_land}/{n} = {p_land/n:.1%}")
print(f"Zensus-Feature vorhanden insgesamt: {(g1['zensus_miete'].notna().sum())/n:.1%}")

# 7) Aufräumen & speichern
drop_cols = ["cand1","cand2","gemeinde_clean","zensus_miete_kreis","zensus_miete_land"]
g1 = g1.drop(columns=[c for c in drop_cols if c in g1.columns])
Path("../data/clean").mkdir(parents=True, exist_ok=True)
g1.to_csv(OUT_PATH, index=False, encoding="utf-8")
print("Gespeichert →", OUT_PATH)


Treffer Gemeinde: 48273/268850 = 18.0%
Fallback Kreis  : 0/268850 = 0.0%
Fallback Land   : 14498/268850 = 5.4%
Zensus-Feature vorhanden insgesamt: 23.3%
Gespeichert → ../data/clean/immo_with_gkz_plus.csv


In [76]:
import re
import pandas as pd

def norm_city(s: pd.Series | str) -> pd.Series | str:
    """sehr einfache Normalisierung für Orts-/Gemeindenamen"""
    if isinstance(s, str):
        s = s.lower().strip()
        s = re.sub(r"\s*\([^)]*\)", "", s)          # Klammerzusätze raus: "Burg (Dith.)" -> "burg"
        s = s.replace("ß", "ss")
        s = s.replace("ä", "ae").replace("ö", "oe").replace("ü", "ue")
        s = re.sub(r"[,.;:/\-]+", " ", s)           # Trennzeichen -> Leerzeichen
        s = re.sub(r"\b(stadt|hansestadt|kreis|landeshauptstadt)\b", "", s)
        s = re.sub(r"\s+", " ", s).strip()
        return s
    else:
        s = s.astype(str)
        return s.apply(norm_city)


In [99]:
import pandas as pd
import numpy as np

# -------- 1) Mapping laden & säubern --------
m = pd.read_csv("../data/ags_gkz.csv", dtype=str, sep=";", encoding="utf-8", names=["ARS", "AGS", "Gemeindename","PLZ", "Ort"])

m

,ARS,AGS,Gemeindename,PLZ,Ort
0,10010000000,1001000,"Flensburg, Stadt",24937,Flensburg
1,10020000000,1002000,"Kiel, Landeshauptstadt",24103,Kiel
2,10030000000,1003000,"Lübeck, Hansestadt",23539,Lübeck
3,10040000000,1004000,"Neumünster, Stadt",24534,Neumünster
4,10510011011,1051011,"Brunsbüttel, Stadt",25541,Brunsbüttel
...,...,...,...,...,...
11728,"1,60775E+11",16077011,Göpfersdorf,4603,Nobitz
11729,"1,60775E+11",16077023,Langenleuba-Niederhain,4603,Nobitz
11730,"1,60775E+11",16077036,Nobitz,4603,Nobitz
11731,"1,60775E+11",16077003,Dobitschen,4626,Schmölln/Thür.


In [100]:
 #Header säubern (BOM, Spaces)
m.columns = [c.encode('utf-8').decode('utf-8-sig').strip() for c in m.columns]

need = {"AGS","Gemeindename","PLZ","Ort"}
missing = need - set(m.columns)
if missing:
    raise ValueError(f"Fehlende Spalten in ags_gkz.csv: {missing}")

m = m.copy()

In [101]:
 #Header säubern (BOM, Spaces)
m.columns = [c.encode('utf-8').decode('utf-8-sig').strip() for c in m.columns]

need = {"AGS","Gemeindename","PLZ","Ort"}
missing = need - set(m.columns)
if missing:
    raise ValueError(f"Fehlende Spalten in ags_gkz.csv: {missing}")

m = m.copy()

# PLZ extrahieren (5-stellig), AGS auf reine Ziffern trimmen
m["PLZ"] = m["PLZ"].astype(str).str.extract(r"(\d{5})", expand=False)
m["AGS"] = m["AGS"].astype(str).str.replace(r"\D", "", regex=True).str.strip()

# Nur valide Zeilen: PLZ vorhanden & AGS vorhanden (damit fliegen Landes-Kopfzeilen raus)
m = m[m["PLZ"].notna() & m["AGS"].str.len().gt(0)].copy()



In [102]:
# Namens-Normalisierung
def norm_city(s):
    if pd.isna(s): 
        return ""
    s = str(s).lower()
    repl = {
        "ß":"ss", "-":" ", "(": "", ")":"", ".":"", ",":""
    }
    for a,b in repl.items():
        s = s.replace(a,b)
    # typische Zusätze entfernen
    s = (s.replace("landeshauptstadt","")
           .replace("kreisfreie stadt","")
           .replace("kreisstadt","")
           .replace("stadt","")
           .replace("gemeinde","")
           .replace("amt ",""))
    return " ".join(s.split()).strip()

m["ort_norm"]      = m["Ort"].map(norm_city)
m["gemeinde_norm"] = m["Gemeindename"].map(norm_city)



In [104]:
def norm_city(s: pd.Series) -> pd.Series:
    s = s.fillna("").astype(str).str.lower().str.strip()
    # häufige Zusätze entfernen
    drop = [
        r",\s*stadt", r",\s*kreis.*", r"\(dith\.\)", r"\(.*?\)", r"\s+landeshauptstadt",
        r"\s+kreis.*", r"\s+kreisfreie stadt", r"\s+hanse.*", r"\s+am see", r"\s+an der .*",
    ]
    for pat in drop:
        s = s.str.replace(pat, "", regex=True)
    # Umlaute / Sonderzeichen
    repl = {
        "ä":"ae","ö":"oe","ü":"ue","ß":"ss","-":" ","/":" ",".":"","!":"","?":"",";":"",
        ":":"","'":"","\"":""
    }
    for k,v in repl.items():
        s = s.str.replace(k, v)
    # Mehrfachspaces → 1 Space
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

m["gemeinde_norm"] = norm_city(m["Gemeindename"])
m["ort_norm"]      = norm_city(m["Ort"])
m[["Gemeindename","Ort","gemeinde_norm","ort_norm"]].head(8)


,Gemeindename,Ort,gemeinde_norm,ort_norm
0,"Flensburg, Stadt",Flensburg,flensburg,flensburg
1,"Kiel, Landeshauptstadt",Kiel,"kiel,",kiel
2,"Lübeck, Hansestadt",Lübeck,"luebeck,",luebeck
3,"Neumünster, Stadt",Neumünster,neumuenster,neumuenster
4,"Brunsbüttel, Stadt",Brunsbüttel,brunsbuettel,brunsbuettel
5,"Heide, Stadt",Heide,heide,heide
6,Averlak,Burg (Dith.),averlak,burg
7,Brickeln,Burg (Dith.),brickeln,burg


In [105]:
# -------- 2) Eindeutige PLZ→AGS bauen --------
plz_unique = (m.groupby("PLZ")["AGS"].nunique() == 1)
plz_unique_idx = plz_unique[plz_unique].index
plz2ags = (m[m["PLZ"].isin(plz_unique_idx)]
           .loc[:, ["PLZ","AGS"]]
           .drop_duplicates())

print("Mapping nach Clean:")
print("  Gesamt-Zeilen:", len(m))
print("  Eindeutige PLZ→AGS:", len(plz2ags))

# Optional: City-Fallback (für später)
city_map = (m.loc[:, ["gemeinde_norm","AGS"]]
              .drop_duplicates())


Mapping nach Clean:
  Gesamt-Zeilen: 10766
  Eindeutige PLZ→AGS: 3296


In [106]:
# PLZ eindeutig?
plz_counts = m.groupby("PLZ")["AGS"].nunique()
plz_unique = plz_counts[plz_counts == 1].index
plz2ags = dict(m.loc[m["PLZ"].isin(plz_unique), ["PLZ","AGS"]].drop_duplicates().values)

# Gemeindename eindeutig?
g_counts = m.groupby("gemeinde_norm")["AGS"].nunique()
g_unique = g_counts[g_counts == 1].index
gname2ags = dict(m.loc[m["gemeinde_norm"].isin(g_unique), ["gemeinde_norm","AGS"]].drop_duplicates().values)

# Ort eindeutig?
o_counts = m.groupby("ort_norm")["AGS"].nunique()
o_unique = o_counts[o_counts == 1].index
oname2ags = dict(m.loc[m["ort_norm"].isin(o_unique), ["ort_norm","AGS"]].drop_duplicates().values)

len(plz2ags), len(gname2ags), len(oname2ags)


(3296, 9029, 3265)

In [108]:
df = pd.read_csv("../data/immo_data.csv", dtype=str)   # wir casten später, hier reicht str
df.columns = df.columns.str.strip()

# PLZ sauber ziehen (5-stellig)
df["plz5"] = df.get("geo_plz", "").astype(str).str.extract(r"(\d{5})", expand=False)

# City-Quelle wählen (regio3 > regio2 > regio1)
city_src = df.get("regio3")
if city_src is None:
    city_src = df.get("regio2")
if city_src is None:
    city_src = df.get("regio1")
df["city_raw"]  = city_src.fillna("")
df["city_norm"] = norm_city(df["city_raw"])

# 1) per PLZ (nur eindeutige PLZs)
df["GKZ"] = df["plz5"].map(plz2ags)

# 2) per Gemeindename (nur eindeutige Namen)
mask = df["GKZ"].isna()
df.loc[mask, "GKZ"] = df.loc[mask, "city_norm"].map(gname2ags)

# 3) per Ort (nur eindeutige Namen)
mask = df["GKZ"].isna()
df.loc[mask, "GKZ"] = df.loc[mask, "city_norm"].map(oname2ags)

# Trefferquote
match_cnt = df["GKZ"].notna().sum()
total = len(df)
print(f"Match-Quote GKZ: {match_cnt}/{total} = {match_cnt/total:.1%}")
df[["plz5","city_raw","city_norm","GKZ"]].head(10)


Match-Quote GKZ: 100767/268850 = 37.5%


,plz5,city_raw,city_norm,GKZ
0,44269,Schüren,schueren,NaN
1,67459,Böhl_Iggelheim,boehl_iggelheim,7338005
2,01097,Äußere_Neustadt_Antonstadt,aeussere_neustadt_antonstadt,NaN
3,09599,Freiberg,freiberg,NaN
4,28213,Neu_Schwachhausen,neu_schwachhausen,NaN
5,24891,Struxdorf,struxdorf,1059082
6,09599,Freiberg,freiberg,NaN
7,28717,St._Magnus,st_magnus,NaN
8,79211,Denzlingen,denzlingen,8316009
9,45888,Bulmke_Hüllen,bulmke_huellen,NaN


In [111]:
# --- A) Eindeutige (PLZ, Name)-Paare nutzen ---

# 1) (PLZ, gemeinde_norm) → AGS, nur wenn eindeutig
pair_counts = m.groupby(["PLZ", "gemeinde_norm"])["AGS"].nunique()
pair_unique = pair_counts[pair_counts == 1].index
pair_df = (
    m.set_index(["PLZ","gemeinde_norm"])
     .loc[pair_unique, ["AGS"]]
     .reset_index()
)

# hier der FIX: Index setzen und Series->dict
pair_map = pair_df.set_index(["PLZ","gemeinde_norm"])["AGS"].to_dict()

mask = df["GKZ"].isna()
pairs = list(zip(df.loc[mask, "plz5"], df.loc[mask, "city_norm"]))
df.loc[mask, "GKZ"] = [pair_map.get(t) for t in pairs]

print("Nach (PLZ, gemeinde_norm):",
      f"{df['GKZ'].notna().sum()}/{len(df)} = {df['GKZ'].notna().mean():.1%}")

# 2) (PLZ, ort_norm) → AGS, nur wenn eindeutig
pair_counts_o = m.groupby(["PLZ","ort_norm"])["AGS"].nunique()
pair_unique_o = pair_counts_o[pair_counts_o == 1].index
pair_df_o = (
    m.set_index(["PLZ","ort_norm"])
     .loc[pair_unique_o, ["AGS"]]
     .reset_index()
)
pair_map_o = pair_df_o.set_index(["PLZ","ort_norm"])["AGS"].to_dict()

mask = df["GKZ"].isna()
pairs_o = list(zip(df.loc[mask, "plz5"], df.loc[mask, "city_norm"]))
df.loc[mask, "GKZ"] = [pair_map_o.get(t) for t in pairs_o]

print("Nach zusätzl. (PLZ, ort_norm):",
      f"{df['GKZ'].notna().sum()}/{len(df)} = {df['GKZ'].notna().mean():.1%}")


Nach (PLZ, gemeinde_norm): 101353/268850 = 37.7%
Nach zusätzl. (PLZ, ort_norm): 101353/268850 = 37.7%


In [112]:
def norm_city(x: str) -> str:
    if pd.isna(x):
        return ""
    x = str(x).lower()
    x = re.sub(r"\([^)]*\)", " ", x)          # Klammerzusätze raus
    x = x.replace(",", " ")
    for bad in [" stadt", " landeshauptstadt", "kreisfreie stadt",
                "gemeinde", "amt", "stadtteil", "bezirk"]:
        x = x.replace(bad, " ")
    x = re.sub(r"[^a-zäöüß0-9 ]+", " ", x)    # Sonderzeichen weg
    x = re.sub(r"\s+", " ", x).strip()
    return x

# auf Immo & Mapping anwenden
df["city_norm"]    = df["city_raw"].map(norm_city)
df["regio3_norm"]  = df.get("regio3", "").map(norm_city)
m["gemeinde_norm"] = m["Gemeindename"].map(norm_city)
m["ort_norm"]      = m["Ort"].map(norm_city)


In [113]:
# Kandidaten je PLZ (nur relevante Spalten, Duplikate raus)
cand = (m[["PLZ","AGS","gemeinde_norm","ort_norm"]]
        .dropna(subset=["PLZ","AGS"])
        .drop_duplicates())

# Tokenizer
def toks(s: str) -> set[str]:
    return set(w for w in s.split() if w)

# Map: PLZ -> Liste von Kandidaten-Dicts
from collections import defaultdict
plz2cands = defaultdict(list)
for row in cand.itertuples(index=False):
    plz2cands[row.PLZ].append({
        "AGS": row.AGS,
        "gemeinde_toks": toks(row.gemeinde_norm),
        "ort_toks": toks(row.ort_norm),
    })

def pick_by_overlap(plz: str, name_norm: str, regio3_norm: str) -> str|None:
    cands = plz2cands.get(plz)
    if not cands:
        return None
    name_toks  = toks(name_norm)
    regio_toks = toks(regio3_norm)
    best_ags, best_score = None, 0
    for c in cands:
        # Overlap zu Gemeinde- und Ort-Token, plus Regio3 als Zusatz
        score = (
            len(c["gemeinde_toks"] & name_toks) +
            len(c["ort_toks"]      & name_toks) +
            0.5 * len((c["gemeinde_toks"] | c["ort_toks"]) & regio_toks)
        )
        if score > best_score:
            best_score, best_ags = score, c["AGS"]
    return best_ags if best_score > 0 else None

mask = df["GKZ"].isna() & df["plz5"].notna()
df.loc[mask, "GKZ"] = [
    pick_by_overlap(plz, nm, rg)
    for plz, nm, rg in zip(df.loc[mask,"plz5"], df.loc[mask,"city_norm"], df.loc[mask,"regio3_norm"])
]

print("Nach Token-Overlap:",
      f'{df["GKZ"].notna().sum()}/{len(df)} = {df["GKZ"].notna().mean():.1%}')


Nach Token-Overlap: 105995/268850 = 39.4%


In [114]:
# 1:1-Lookup (regio3_norm -> eindeutige AGS), nur wenn eindeutig
r_counts = m.groupby("gemeinde_norm")["AGS"].nunique()
r_unique = r_counts[r_counts == 1].index
r_map = (m[m["gemeinde_norm"].isin(r_unique)]
         .drop_duplicates(subset=["gemeinde_norm"])
         .set_index("gemeinde_norm")["AGS"]
         .to_dict())

mask = df["GKZ"].isna() & df["regio3_norm"].notna()
df.loc[mask, "GKZ"] = df.loc[mask, "regio3_norm"].map(r_map)

print("Nach Regio3-Direkttreffer:",
      f'{df["GKZ"].notna().sum()}/{len(df)} = {df["GKZ"].notna().mean():.1%}')


Nach Regio3-Direkttreffer: 107775/268850 = 40.1%


In [115]:
import re

# --- Hilfsspalten im IMMO-DF: Kreis- und BL-Codes ---
# Versuch 1: 5-stelliger Kreisschlüssel (aus geo_krs / regio2 – was immer näher dran ist)
def extract_kreis_code(x):
    if pd.isna(x):
        return None
    s = str(x)
    # häufig stehen vorne/irgendwo 5-stellige Kreiskennziffern
    m = re.search(r"(\d{5})", s)
    return m.group(1) if m else None

if "geo_krs" in df.columns and df["geo_krs"].notna().any():
    df["kreis5"] = df["geo_krs"].map(extract_kreis_code)
elif "regio2" in df.columns:
    df["kreis5"] = df["regio2"].map(extract_kreis_code)
else:
    df["kreis5"] = None

# Versuch 2: BL-Code aus Bundeslandnamen (regio1 / geo_bln)
BL2CODE = {
    "schleswig-holstein":"01","hamburg":"02","niedersachsen":"03","bremen":"04",
    "nordrhein-westfalen":"05","hessen":"06","rheinland-pfalz":"07","baden-württemberg":"08",
    "bayern":"09","saarland":"10","berlin":"11","brandenburg":"12","mecklenburg-vorpommern":"13",
    "sachsen":"14","sachsen-anhalt":"15","thüringen":"16",
}
def bl_code_from_name(x):
    if pd.isna(x): return None
    name = re.sub(r"\s+"," ",str(x).strip().lower())
    return BL2CODE.get(name)

src_bl = "regio1" if "regio1" in df.columns else ("geo_bln" if "geo_bln" in df.columns else None)
df["bl2"] = df[src_bl].map(bl_code_from_name) if src_bl else None

# --- Kandidaten aus Mapping vorbereiten (wie zuvor) ---
cand = (m[["PLZ","AGS","gemeinde_norm","ort_norm"]]
        .dropna(subset=["PLZ","AGS"])
        .drop_duplicates())

def toks(s: str) -> set[str]:
    return set(w for w in str(s).split() if w)

# Index: PLZ -> Kandidaten (mit vorab gebildeten Token + Codes)
from collections import defaultdict
plz2cands = defaultdict(list)
for row in cand.itertuples(index=False):
    ags = str(row.AGS)
    kreis5 = ags[:5] if len(ags) >= 5 else None
    bl2    = ags[:2] if len(ags) >= 2 else None
    plz2cands[row.PLZ].append({
        "AGS": ags,
        "kreis5": kreis5,
        "bl2": bl2,
        "gemeinde_toks": toks(row.gemeinde_norm),
        "ort_toks": toks(row.ort_norm),
        "label": f"{row.gemeinde_norm or ''}|{row.ort_norm or ''}",
    })

def pick_by_overlap_filtered(plz, name_norm, regio3_norm, kreis5_hint, bl2_hint):
    cands = plz2cands.get(plz)
    if not cands:
        return None

    # 1) Kreisfilter, 2) sonst BL-Filter, 3) sonst alle
    if kreis5_hint:
        pool = [c for c in cands if c["kreis5"] == kreis5_hint]
        if not pool:  # nichts gefunden? auf BL runterfallen
            pool = [c for c in cands if bl2_hint and c["bl2"] == bl2_hint]
    elif bl2_hint:
        pool = [c for c in cands if c["bl2"] == bl2_hint]
    else:
        pool = cands

    if not pool:
        pool = cands  # als letzte Chance: alle

    name_toks  = toks(name_norm)
    regio_toks = toks(regio3_norm)

    best, best_score = None, 0
    for c in pool:
        score = (
            len(c["gemeinde_toks"] & name_toks) +
            len(c["ort_toks"]      & name_toks) +
            0.5 * len((c["gemeinde_toks"] | c["ort_toks"]) & regio_toks)
        )
        if score > best_score:
            best_score, best = score, c

    return best["AGS"] if best and best_score > 0 else None

mask = df["GKZ"].isna() & df["plz5"].notna()
df.loc[mask, "GKZ"] = [
    pick_by_overlap_filtered(plz, nm, rg, k5, bl)
    for plz, nm, rg, k5, bl in zip(
        df.loc[mask,"plz5"],
        df.loc[mask,"city_norm"],
        df.loc[mask,"regio3_norm"],
        df.loc[mask,"kreis5"],
        df.loc[mask,"bl2"],
    )
]

print("Nach Kreis/BL-Filter + Overlap:",
      f'{df["GKZ"].notna().sum()}/{len(df)} = {df["GKZ"].notna().mean():.1%}')


Nach Kreis/BL-Filter + Overlap: 107775/268850 = 40.1%


In [117]:
try:
    from rapidfuzz import fuzz
    HAVE_FUZZ = True
except Exception:
    HAVE_FUZZ = False

def pick_by_fuzzy(plz, name_norm, regio3_norm, kreis5_hint, bl2_hint, threshold=90):
    if not HAVE_FUZZ:
        return None
    cands = plz2cands.get(plz)
    if not cands:
        return None

    # gleiche Filterlogik
    if kreis5_hint:
        pool = [c for c in cands if c["kreis5"] == kreis5_hint] or \
               [c for c in cands if bl2_hint and c["bl2"] == bl2_hint]
    elif bl2_hint:
        pool = [c for c in cands if c["bl2"] == bl2_hint]
    else:
        pool = cands
    if not pool:
        pool = cands

    target = " ".join(w for w in [name_norm, regio3_norm] if w)
    best, best_score = None, -1
    for c in pool:
        cand_label = c["label"]
        score = fuzz.token_sort_ratio(target, cand_label)
        if score > best_score:
            best, best_score = c, score

    return best["AGS"] if best and best_score >= threshold else None

if HAVE_FUZZ:
    mask = df["GKZ"].isna() & df["plz5"].notna()
    df.loc[mask, "GKZ"] = [
        pick_by_fuzzy(plz, nm, rg, k5, bl)
        for plz, nm, rg, k5, bl in zip(
            df.loc[mask,"plz5"],
            df.loc[mask,"city_norm"],
            df.loc[mask,"regio3_norm"],
            df.loc[mask,"kreis5"],
            df.loc[mask,"bl2"],
        )
    ]
    print("Nach Fuzzy-Fallback:",
          f'{df["GKZ"].notna().sum()}/{len(df)} = {df["GKZ"].notna().mean():.1%}')
else:
    print("RapidFuzz nicht installiert – Fuzzy-Fallback übersprungen.")


Nach Fuzzy-Fallback: 107775/268850 = 40.1%


In [120]:
matched = df["GKZ"].notna().sum()
print(f"Gesamt-Match-Quote GKZ: {matched}/{len(df)} = {matched/len(df):.1%}")

df.to_csv("../data/clean/immo_with_gkz1.csv", index=False, encoding="utf-8")
df[df["GKZ"].isna()].sample(1000, random_state=42).to_csv(
    "../data/clean/immo_gkz_unmatched_sample.csv", index=False, encoding="utf-8"
)
print("→ Dateien aktualisiert.")


Gesamt-Match-Quote GKZ: 107775/268850 = 40.1%
→ Dateien aktualisiert.


In [124]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

# -----------------------------
# 0) Pfade
# -----------------------------
MAP_CSV = "../data/ags_gkz.csv"              # deine PLZ/Ort/ARS/AGS Mapping-Datei (mit ;)
IMMO_IN  = "../data/clean/immo_clean8.csv"   # deine aktuelle bereinigte Immo-CSV
IMMO_OUT = "../data/clean/immo_with_ars.csv"
UNMATCH  = "../data/clean/immo_ars_unmatched_sample.csv"

# -----------------------------
# 1) Helper: Ortsnamen normalisieren (sehr sanft)
# -----------------------------
def norm_city(s: pd.Series) -> pd.Series:
    s = s.astype(str).fillna("")
    # Lowercase
    s = s.str.lower()
    # Klammern/Kommas/Punkte entfernen, Bindestrich zu Leerzeichen
    s = s.str.replace(r"[\(\)\.,]", "", regex=True)
    s = s.str.replace("-", " ", regex=False)
    # typische Zusätze entfernen
    remove = [
        "stadt", "kreis", "hansestadt", "landeshauptstadt", "amtsfreie stadt",
        "amt ", "verbandsgemeinde", "verband", "gemeinde"
    ]
    for w in remove:
        s = s.str.replace(rf"\b{re.escape(w)}\b", "", regex=True)
    # mehrfachspaces kürzen & trimmen
    s = s.str.replace(r"\s+", " ", regex=True).str.strip()
    return s

# -----------------------------
# 2) Mapping laden & sauber ziehen
# -----------------------------
m = pd.read_csv(MAP_CSV, sep=";", dtype=str, encoding="utf-8", names=["ARS", "AGS", "Gemeindename", "PLZ", "Ort"])
need = ["ARS", "AGS", "Gemeindename", "PLZ", "Ort"]
missing = [c for c in need if c not in m.columns]
if missing:
    raise ValueError(f"In {MAP_CSV} fehlen Spalten: {missing}")

m = m.copy()
m["PLZ"] = m["PLZ"].astype(str).str.extract(r"(\d{5})", expand=False)
m = m[m["PLZ"].notna()].copy()

m["ARS"] = m["ARS"].astype(str).str.strip()
m["AGS"] = m["AGS"].astype(str).str.strip()

m["gemeinde_norm"] = norm_city(m["Gemeindename"])
m["ort_norm"]      = norm_city(m["Ort"])

# --- Lookups ---
# (a) (PLZ, gemeinde_norm) → ARS nur wenn eindeutig
pair_counts = m.groupby(["PLZ", "gemeinde_norm"])["ARS"].nunique()
pair_unique = pair_counts[pair_counts == 1].index
pair_map = (
    m.set_index(["PLZ","gemeinde_norm"])
     .loc[pair_unique, ["ARS"]]
     .reset_index()
     .set_index(["PLZ","gemeinde_norm"])["ARS"]
     .to_dict()
)

# (b) PLZ → ARS nur wenn eindeutig
plz_counts = m.groupby("PLZ")["ARS"].nunique()
plz_unique = plz_counts[plz_counts == 1].index
plz_map = m[m["PLZ"].isin(plz_unique)].drop_duplicates("PLZ").set_index("PLZ")["ARS"].to_dict()

# (c) Name (Gemeinde) → ARS nur wenn eindeutig
name_counts = m.groupby("gemeinde_norm")["ARS"].nunique()
name_unique = name_counts[name_counts == 1].index
name_map = m[m["gemeinde_norm"].isin(name_unique)].drop_duplicates("gemeinde_norm").set_index("gemeinde_norm")["ARS"].to_dict()

# (d) Ort → ARS nur wenn eindeutig (zusätzlicher Pfeil)
ort_counts = m.groupby("ort_norm")["ARS"].nunique()
ort_unique = ort_counts[ort_counts == 1].index
ort_map = m[m["ort_norm"].isin(ort_unique)].drop_duplicates("ort_norm").set_index("ort_norm")["ARS"].to_dict()

print(f"Mapping-Größen → pair:{len(pair_map)}  plz:{len(plz_map)}  name:{len(name_map)}  ort:{len(ort_map)}")

# -----------------------------
# 3) Immo laden & ARS zuordnen
# -----------------------------
df = pd.read_csv(IMMO_IN, dtype=str)

# PLZ holen (geo_plz bevorzugt, sonst zip_code)
if "geo_plz" in df.columns:
    df["plz5"] = df["geo_plz"].astype(str).str.extract(r"(\d{5})", expand=False)
elif "zip_code" in df.columns:
    df["plz5"] = df["zip_code"].astype(str).str.extract(r"(\d{5})", expand=False)
else:
    df["plz5"] = np.nan

# Städtenamenquelle wählen (regio3 > regio2 > regio1 > city)
city_source = None
for c in ["regio3", "regio2", "regio1", "city", "City", "ort"]:
    if c in df.columns:
        city_source = c
        break

if city_source is not None:
    df["city_norm"] = norm_city(df[city_source])
else:
    df["city_norm"] = ""

df["ARS"] = np.nan  # Zielspalte

def fill_where(mask, series, mapping):
    # map auf eine Series und nur dort übernehmen, wo mask True ist
    mapped = series.map(mapping)
    return np.where(mask & series.notna() & mapped.notna(), mapped, np.nan)

# (1) (PLZ, Name) eindeutig
mask = df["ARS"].isna()
pairs = list(zip(df.loc[mask, "plz5"].fillna(""), df.loc[mask, "city_norm"].fillna("")))
df.loc[mask, "ARS"] = pd.Series(pairs, index=df.index[mask]).map(pair_map)

# (2) PLZ eindeutig
mask = df["ARS"].isna()
df.loc[mask, "ARS"] = df.loc[mask, "plz5"].map(plz_map)

# (3) Name eindeutig (Gemeinde)
mask = df["ARS"].isna()
df.loc[mask, "ARS"] = df.loc[mask, "city_norm"].map(name_map)

# (4) Ort eindeutig (zweite Namensquelle)
mask = df["ARS"].isna()
df.loc[mask, "ARS"] = df.loc[mask, "city_norm"].map(ort_map)

matched = df["ARS"].notna().sum()
total   = len(df)
print(f"Gesamt-Match-Quote ARS: {matched}/{total} = {matched/total:.1%}")

# -----------------------------
# 4) Speichern + Sample der Nicht-Treffer
# -----------------------------
Path(IMMO_OUT).parent.mkdir(parents=True, exist_ok=True)
df.to_csv(IMMO_OUT, index=False, encoding="utf-8")
print("→ Gespeichert:", IMMO_OUT)

unmatched = df[df["ARS"].isna()].sample(min(200, df["ARS"].isna().sum()), random_state=42)
unmatched.to_csv(UNMATCH, index=False, encoding="utf-8")
print("→ Unmatched-Sample:", UNMATCH)


Mapping-Größen → pair:10620  plz:3456  name:9214  ort:3477


C:\Users\Alex\AppData\Local\Temp\ipykernel_36308\1807206166.py:118: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[nan nan nan ... nan nan nan]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[mask, "ARS"] = pd.Series(pairs, index=df.index[mask]).map(pair_map)


Gesamt-Match-Quote ARS: 101662/268850 = 37.8%
→ Gespeichert: ../data/clean/immo_with_ars.csv
→ Unmatched-Sample: ../data/clean/immo_ars_unmatched_sample.csv


In [128]:
import pandas as pd
import re

# --- kleine Normalisierung für Städtenamen ---
def norm_city(x: str) -> str:
    if pd.isna(x):
        return None
    x = str(x)
    x = x.replace("ß", "ss")
    x = re.sub(r"[(),.]", " ", x)     # Klammern/Punkte/Kommas raus
    x = re.sub(r"\s+", " ", x).strip().lower()
    # Suffixe wie ", stadt", ", kreis", "kreis ..." entfernen
    x = re.sub(r"\b(stadt|kreis|landeshauptstadt|hansestadt)\b", "", x)
    x = re.sub(r"\s+", " ", x).strip()
    return x

# --- 1) Zensus laden & eindeutige Namen -> GKZ ---
z = pd.read_csv("../data/clean/zensus_0004_clean.csv", dtype=str)
z["GKZ"] = z["GKZ"].str.replace(r"\D", "", regex=True).str.zfill(12)
z["name_norm"] = z["Gemeindename"].map(norm_city)

# nur Namen, die GENAU EINMAL im Zensus vorkommen
counts = z.groupby("name_norm")["GKZ"].nunique()
unique_names = counts[counts == 1].index
name2gkz = dict(z[z["name_norm"].isin(unique_names)].set_index("name_norm")["GKZ"])

print(f"Eindeutige Zensus-Namen: {len(name2gkz):,}")

# --- 2) Immo laden ---
df = pd.read_csv("../data/clean/immo_clean8.csv", dtype=str)

# Kandidatenname aus regio3 (feiner) und regio2 (gröber) bilden
cand3 = df.get("regio3", pd.Series(index=df.index, dtype="object")).map(norm_city)
cand2 = df.get("regio2", pd.Series(index=df.index, dtype="object")).map(norm_city)

# zuerst regio3, dann regio2
gkz_by_name = cand3.map(name2gkz)
mask = gkz_by_name.isna()
gkz_by_name.loc[mask] = cand2[mask].map(name2gkz)

# nur leere ARS/GKZ befüllen (falls du ARS schon hast, nimm eine extra Spalte)
if "ARS" not in df.columns:
    df["ARS"] = pd.NA
fill_mask = df["ARS"].isna() & gkz_by_name.notna()
df.loc[fill_mask, "ARS"] = gkz_by_name[fill_mask]

print("Neu gefüllte via eindeutiger Name-Match:", int(fill_mask.sum()))
print("Gesamt mit ARS:", int(df["ARS"].notna().sum()))

df.to_csv("../data/clean/immo_with_ars2.csv", index=False, encoding="utf-8")
print("→ Gespeichert: data/clean/immo_with_ars2.csv")


Eindeutige Zensus-Namen: 9,673


Neu gefüllte via eindeutiger Name-Match: 211455
Gesamt mit ARS: 211455
→ Gespeichert: data/clean/immo_with_ars2.csv


Was haben wir entfernt?

regio1, regio2, regio3, geo_plz, geo_krs, geo_bln → raus, weil ARS der Standortschlüssel ist.

Freitext & selten genutztes (z. B. description) brauchst du fürs RF-Baseline nicht.

In [131]:
import pandas as pd
import numpy as np

# 1) Laden
df = pd.read_csv("../data/clean/immo_with_ars2.csv", dtype=str)

# 2) ARS sauber halten (12-stellig) und nur Zeilen mit ARS behalten
df["ARS"] = df["ARS"].str.replace(r"\D", "", regex=True).str.zfill(12)
df = df[df["ARS"].notna() & (df["ARS"] != "")].copy()
print("Zeilen mit ARS:", len(df))

# 3) Typen und Umbenennungen (so wie wir es im EDA gemacht hatten)
#    → falls bei dir bereits passiert, ist das hier idempotent.
rename_map = {
    "baseRent": "rent_cold",
    "livingSpace": "area_sqm",
    "noRooms": "rooms",
}
df = df.rename(columns=rename_map)

# numerische Spalten erzwingen (fehltolerant)
for col in ["rent_cold","area_sqm","rooms","floor","pricetrend","serviceCharge","totalRent","yearConstructed"]:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Booleans nach 0/1 (falls noch nicht erledigt)
for col in ["garden","lift","hasKitchen","balcony","cellar"]:
    if col in df.columns:
        df[col] = df[col].astype(str).str.lower().isin(["1","true","yes","ja"]).astype(int)

# 4) Spaltenauswahl für das Training
#    – minimal, aber sinnvoll (Standort, Größe, Zimmer, Etage, Ausstattung, Heizung, Dekade)
keep_cols = [
    "ARS",               # Schlüssel für Zensus-Join / Standort
    "rent_cold",         # TARGET
    "area_sqm", "rooms", "floor",
    "pricetrend", "serviceCharge",
    "garden", "lift", "hasKitchen",
    "typeOfFlat",        # kategorial
    "heatingType_clean", # kategorial (aus heatingType abgeleitet)
    "periode_0005",      # Dekaden-Bucket aus yearConstructed (für 0005-Join)
]
# Nimm nur Spalten, die wirklich existieren
keep_cols = [c for c in keep_cols if c in df.columns]
df = df[keep_cols].copy()

# 5) Einfache Imputation/Filter (saubere Baseline)
# floor → häufigster Wert (Mode)
if "floor" in df.columns:
    mode_floor = df["floor"].mode(dropna=True)
    if len(mode_floor):
        df["floor"] = df["floor"].fillna(mode_floor.iloc[0])

# serviceCharge → 0 (deine Vorgabe)
if "serviceCharge" in df.columns:
    df["serviceCharge"] = df["serviceCharge"].fillna(0)

# pricetrend → Mittelwert
if "pricetrend" in df.columns:
    df["pricetrend"] = df["pricetrend"].fillna(df["pricetrend"].mean())

# Kategoriale fehlende Werte → "unknown"
for col in ["typeOfFlat","heatingType_clean","periode_0005"]:
    if col in df.columns:
        df[col] = df[col].fillna("unknown").astype("string")

# Plausibilitätsfilter (wie beim Training): Fläche & Miete
if "area_sqm" in df.columns:
    df = df[(df["area_sqm"] > 10) & (df["area_sqm"] < 400)]
if "rent_cold" in df.columns:
    df = df[(df["rent_cold"] > 200) & (df["rent_cold"] < 10000)]

print("Nach Clean:", df.shape)

# 6) Speichern
out_path = "../data/clean/immo_train_ready.csv"
df.to_csv(out_path, index=False, encoding="utf-8")
print("→ Gespeichert:", out_path)
print("Finale Spalten:", list(df.columns))


Zeilen mit ARS: 211455
Nach Clean: (205543, 13)
→ Gespeichert: ../data/clean/immo_train_ready.csv
Finale Spalten: ['ARS', 'rent_cold', 'area_sqm', 'rooms', 'floor', 'pricetrend', 'serviceCharge', 'garden', 'lift', 'hasKitchen', 'typeOfFlat', 'heatingType_clean', 'periode_0005']


In [141]:
# 1) Zensus 0005 laden
z5 = pd.read_csv("../data/clean/zensus_0005_clean1.csv", sep=",", dtype=str, encoding="utf-8")

# numerische Werte erzwingen
for c in z5.columns:
    if c not in ["GKZ","Gemeindename"]:
        z5[c] = pd.to_numeric(z5[c], errors="coerce")

# 2) Dekaden-Faktoren bauen
dekade_cols = ["vor_1919","1919_1949","1950_1959","1970_1979",
               "1980_1989","2000_2009","2010_2015","2016_plus"]
for c in dekade_cols:
    z5[f"factor_{c}"] = z5[c] / z5["Insgesamt"]

# 3) Immo laden
immo = pd.read_csv("../data/clean/immo_train_ready.csv", dtype=str)
for c in ["rent_cold","area_sqm","rooms","floor","pricetrend","serviceCharge"]:
    if c in immo.columns:
        immo[c] = pd.to_numeric(immo[c], errors="coerce")

# 4) Merge über ARS
# In 0005 heißt die Spalte "GKZ" → das ist dein ARS
z5.rename(columns={"GKZ":"ARS"}, inplace=True)
immo["ARS"] = immo["ARS"].astype(str).str.strip()
z5["ARS"]   = z5["ARS"].astype(str).str.strip()

joined = immo.merge(z5, how="left", on="ARS")

# 5) passendes Zensus-Dekadenfeld auswählen
pmap = {
    "vor_1919": "vor_1919",
    "1919_1949": "1919_1949",
    "1950_1959": "1950_1959",
    "1970_1979": "1970_1979",
    "1980_1989": "1980_1989",
    "2000_2009": "2000_2009",
    "2010_2015": "2010_2015",
    "2016_plus": "2016_plus",
}

def pick_decade_value(row):
    key = pmap.get(str(row["periode_0005"]))
    if not key or pd.isna(row.get(key)):
        return np.nan
    return row[key]

def pick_decade_factor(row):
    key = pmap.get(str(row["periode_0005"]))
    fcol = f"factor_{key}" if key else None
    if not fcol or pd.isna(row.get(fcol)):
        return np.nan
    return row[fcol]

joined["zensus_miete_decade"]  = joined.apply(pick_decade_value, axis=1)
joined["zensus_factor_decade"] = joined.apply(pick_decade_factor, axis=1)
joined.rename(columns={"Insgesamt":"zensus_miete_total"}, inplace=True)

# 6) Speichern
out_cols = [
    "ARS","rent_cold","area_sqm","rooms","floor","pricetrend","serviceCharge",
    "garden","lift","hasKitchen","typeOfFlat","heatingType_clean","periode_0005",
    "zensus_miete_total","zensus_miete_decade","zensus_factor_decade"
]
train_join = joined[out_cols].copy()

print("Coverage decade:", train_join["zensus_miete_decade"].notna().mean()*100, "%")
train_join.to_csv("../data/clean/immo_train_joined.csv", index=False, encoding="utf-8")
print("→ Gespeichert: ../data/clean/immo_train_joined.csv")


Coverage decade: 55.093451892778525 %
→ Gespeichert: ../data/clean/immo_train_joined.csv
